# INSERE EVASÃO DE TODOS OS ALUNOS


#### IMPORTAÇÕES

In [ ]:
import pandas as pd     
import pickle # Carregar os modelos criados para cada machine
#import de conexão com o seu Banco de Dados

#### PREPARA OS DADOS, ALTERA VALORES NULOS PARA "SEM VALOR"; FEATURES DE SIM E NÃO PARA BINÁRIAS; ADICIONANDO OS VALORES NUMÉRICOS

In [ ]:

def preparar_dados(df):
    colunas = ['EVADIDO','DP','INADIMPLENTE','QTD_MESES_INADI','CURSO','QTD_DP',
               'MEDIA_DP_POR_INADI','PERFIL_RISCO','MEDIA_COBRANCAS','QTD_PERIODOS']

    df[colunas] = df[colunas].fillna("SEM VALOR").astype(str).apply(lambda x: x.str.strip().str.upper())

   
    df['DP'] = df['DP'].apply(lambda x: 1 if x == 'S' else 0)
    df['EVADIDO'] = df['EVADIDO'].apply(lambda x: 1 if x == 'S' else 0)
    df['INADIMPLENTE'] = df['INADIMPLENTE'].apply(lambda x: 1 if x == 'S' else 0)


    numericas = ['QTD_MESES_INADI', 'QTD_DP', 'MEDIA_DP_POR_INADI', 'MEDIA_COBRANCAS',
                 'PERFIL_RISCO', 'QTD_PERIODOS', 'CURSO']
    for col in numericas:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    return df

## INSERÇÃO NA TABELA

In [ ]:
# conexão com o banco de dados
def processar_modelo (nome_tabela_origem,nome_tabela_destino, modelo_pickle_path ):
    conn = Open_Connection_DataBase(Server="seu_servidor_bd", 
                                    DataBase="sua_database", 
                                    UserName="seu_username", 
                                    Password="sua_senha_bd", 
                                    Driver="driver_do_seubd") # Exemplo de conexão com o banco de dados (Sql Server)
    
    df = pd.read_sql(f"SELECT * FROM {nome_tabela_origem}", conn)
    df = preparar_dados(df)




#Carrega o modelo do arquivo pickle(DE CADA MACHINE)
    with open(modelo_pickle_path, 'rb') as f:
        modelo_carregado  = pickle.load(f)

    modelo = modelo_carregado['modelo'] if isinstance(modelo_carregado, dict) else modelo_carregado




    features = ['CURSO', 'INADIMPLENTE', 'DP', 'QTD_DP', 'QTD_MESES_INADI',
                'MEDIA_DP_POR_INADI','MEDIA_COBRANCAS','PERFIL_RISCO','QTD_PERIODOS']
    X = df[features]



    df['PROB_EVASAO'] = modelo.predict_proba(X)[:, 1]





# Cursor para inserir alunos na tabela do sql
    cursor = conn.cursor()
    for index, row in df.iterrows():
        cursor.execute(f"""
            INSERT INTO {nome_tabela_destino} (ALUNO, PROB_EVASAO, DATA_DE_INSERCAO)
            VALUES (?, ?, GETDATE())
        """, row['ALUNO'], row['PROB_EVASAO'])

    conn.commit()
    cursor.close()
    conn.close()



### TODAS AS TABELAS DE INSERÇÃO, AS TABELAS DESTINO, E OS PICKLES DE CADA MODELO
processar_modelo("ML_EVASAO_PRESENCIAL", "ML_PREDICAO_PRESENCIAL", "modelo_lgb_com_threshold_034_real.pkl")
processar_modelo("ML_EVASAO_HIBRIDO", "ML_PREDICAO_HIBRIDO", "modelo_lgb_com_threshold_05_real.pkl")
processar_modelo("ML_EVASAO_EAD", "ML_PREDICAO_EAD", "modelo_lgb_com_threshold_0.447_real.pkl")